In [1]:
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
import math
from itertools import product
import argparse
import sys
from utils_gruCS import *
import calendar
import time

C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vipuser\.conda\envs\tensorflow-gpu\lib\s

In [2]:
def process_eachseq(seq,pssmfile,mask_seq,new_pssms):
    if os.path.exists(pssmfile):  #如果pssm文件存在
        print("found " + pssmfile + "\n")  #输出找到pssm文件+换行
        pssm = readPSSM(pssmfile)  #读取pssm文件
    else:  #否则
        print("using Blosum62\n")  #输出使用Blosum62+换行
        pssm = convertSampleToBlosum62(seq)  #把Blosum62矩阵当作pssm用
        #pssm = convertSampleToCBOW(seq)
    pssm = pssm.astype(float)  #对pssm的数据类型转换为浮点型
    PhyChem = convertSampleToPhysicsVector_pca(seq)  #将样本转化为物理向量
    pssm = np.concatenate((PhyChem, pssm), axis=1)  #物化指标和pssm对应行进行数组拼接
    seql = len(seq)   #序列长度  
    if seql <= 1000:  #如果序列长度小于等于1000
        padnum = 1000 - seql  #pad大小为1000-序列长度
        padmatrix = np.zeros([padnum, 25])  #pad矩阵为行数为padnum，列数为25的全0矩阵，即用0填充不足的地方
        pssm = np.concatenate((pssm, padmatrix), axis=0)  #物化指标和pssm进行数组拼接 
        new_pssms.append(pssm)  #新的pssm空列表中添加pssm矩阵
        mask_seq.append(gen_mask_mat(seql, padnum))  #mask序列空列表添加gen_mask矩阵，序列长度为行数，padnum为列数？？？
    else:  #如果序列长度大于1000
        pssm = np.concatenate((pssm[0:500, :], pssm[seql - 500:seql, :]), axis=0)  #pssm矩阵为前500行和后500行矩阵的拼接
        new_pssms.append(pssm)  #新的pssm空列表中添加pssm矩阵
        mask_seq.append(gen_mask_mat(1000, 0))  #mask序列空列表添加1000行0列的？？？gen_mask矩阵

In [3]:
def endpad(seqfile, labelfile, pssmdir="", npzfile = ""): #定义endpad(序列文件，标签文件，pssm路径，npz文件)
    if not os.path.exists(npzfile):  #如果npz文件不存在，建立新的pssm空列表，标签空列表，mask序列空列表，id空列表
        new_pssms = []
        labels = []
        mask_seq = []
        ids=[]
        seqs=[]
        f = open(seqfile, "r")  #f为打开序列文件
        f2 = open(labelfile, "r")  #f2为打开标签文件
        line = f.readline()  #读取序列文件的第一行
        while line != '':
            pssmfile = pssmdir + line[1:].strip() + "_pssm.txt"  #pssm文件名=pssm地址+id名+_pssm.txt
            if line[0] == '>':  #如果该行第一个字符为>
                id = line.strip()[1:]  #id为去掉>的字符
                ids.append(id)   #在id空列表中添加id
            label = f2.readline().strip()  #标签为f2（标签文件）中去掉首尾空格的内容
            labels.append(label)  #在标签空列表中添加标签
            seq = f.readline().strip()  #第一次seq为第2行的内容，实际seq为>行的下一行
            #seql = len(seq)   #序列长度  
            process_eachseq(seq,pssmfile,mask_seq,new_pssms)
            line = f.readline()  #继续读取下一行，即>行
        x = np.array(new_pssms)  #把new_pssms列表变为数组，赋给x
        y = [convertlabels_to_categorical(i) for i in labels]  #把标签列表转化为类别(i)
        y = np.array(y)  #再把类别转化为数组
        mask = np.array(mask_seq)  #把mask_seq（标注的序列？）转化为数组
        np.savez(npzfile, x=x, y=y, mask=mask, ids=ids)  #保存多个数组到同一个文件中,保存格式是.npz
        return [x, y, mask,ids]  #返回pssm矩阵，类别，标注序列，名字id
    else:  #如果上述都存在，直接转化为数组
        mask = np.load(npzfile)['mask']
        x = np.load(npzfile)['x']
        y = np.load(npzfile)['y']
        ids=np.load(npzfile)['ids']
        return [x, y, mask,ids]

In [4]:
def train_MULocDeep(lv1_dir,lv2_dir,pssm_dir,output_dir,foldnum):
    # get small data
    [train_x, train_y, train_mask, train_ids] = endpad(
        lv2_dir+"lv2_train_fold" + str(foldnum) + "_seq",
        lv2_dir+"lv2_train_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv2_train_fold"+str(foldnum)+"_seq.npz")
    [val_x, val_y, val_mask,val_ids] = endpad(
        lv2_dir+"lv2_val_fold" + str(foldnum) + "_seq",
        lv2_dir+"lv2_val_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv2_val_fold"+str(foldnum)+"_seq.npz")

    # get big data 训练10分类的多分类
    [train_x_big, train_y_big, train_mask_big, train_ids_big] = endpad(
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv1_train_fold" + str(foldnum) + "_seq.npz")

    [val_x_big, val_y_big, val_mask_big, val_ids_big] = endpad(
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_lab",
        pssm_dir,
        "D:/mulocdeep/mul_data/lv1_val_fold" + str(foldnum) + "_seq.npz")

    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model_big, model_small = singlemodel(train_x)  #模型为singlemodel

    filepath_acc_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_acc_small_lv2 = output_dir+"fold" + str(
        foldnum) + "_small_lv2_acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_loss_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_loss-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

    filepath_loss_small_lv2 = output_dir+"fold" + str(
        foldnum) + "_small_lv2_loss-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"


    checkpoint_acc_big_lev1 = ModelCheckpoint(filepath_acc_big_lv1, monitor='val_accuracy', save_best_only=True,
                                          mode='max',
                                          save_weights_only=True, verbose=1)

    checkpoint_acc_small_lev2 = ModelCheckpoint(filepath_acc_small_lv2, monitor='val_lev2_accuracy', save_best_only=True,
                                          mode='max',
                                          save_weights_only=True, verbose=1)
    
    checkpoint_loss_big_lev1 = ModelCheckpoint(filepath_loss_big_lv1, monitor='val_loss', save_best_only=True,
                                          mode='min',
                                          save_weights_only=True, verbose=1)
    
    checkpoint_loss_small_lev2 = ModelCheckpoint(filepath_loss_small_lv2, monitor='val_lev2_loss', save_best_only=True,
                                          mode='min',
                                          save_weights_only=True, verbose=1)
    
    
    for i in range(20):
        # train small model
        print("epoch "+str(i)+"\n")
        '''fitHistory_batch_small = model_small.fit([train_x, train_mask.reshape(-1, 1000, 1)],
                                                 [train_y,getTrue4out1(train_y)],
                                                 batch_size=batch_size, epochs=1,
                                                 validation_data=(
                                                 [val_x, val_mask.reshape(-1, 1000, 1)], [val_y,getTrue4out1(val_y)]),
                                                 callbacks=[checkpoint_acc_small_lev2,checkpoint_loss_small_lev2],verbose=1)'''
        
        # train big model  
        fitHistory_batch_big = model_big.fit([train_x_big, train_mask_big.reshape(-1, 1000, 1)],
                                             [getTrue4out1(train_y_big)],  #为何大模型没有train_y_big
                                             batch_size=batch_size, epochs=1,  #等于1？？
                                             validation_data=(
                                             [val_x_big, val_mask_big.reshape(-1, 1000, 1)], [getTrue4out1(val_y_big)]),  #也没有val_y_big
                                             callbacks=[checkpoint_acc_big_lev1,checkpoint_loss_big_lev1], verbose=1)


In [5]:
def train_var(input_var,pssm_dir,output_dir,foldnum):
    # get small data
    [train_x,train_y,train_mask,train_ids]=endpad(input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_seq",
                                        input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_label",
                                        pssm_dir,
                                        "D:/deeploc/deeploc_40nr_8folds/train_fold"+str(foldnum)+"_seq.npz")
    [val_x,val_y,val_mask,val_ids]=endpad(input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_seq",
                                  input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_label",
                                  pssm_dir,
                                  "D:/deeploc/deeploc_40nr_8folds/val_fold"+str(foldnum)+"_seq.npz")
    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model = var_model(train_x)   #这里的模型是var_model

    filepath_acc = output_dir+"fold" + str(foldnum) + "acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
    checkpoint_acc = ModelCheckpoint(filepath_acc, monitor='val_accuracy', save_best_only=True, mode='max',
                                 save_weights_only=True, verbose=1)
    fitHistory_batch = model.fit([train_x,train_mask.reshape(-1,1000,1)],getTrue4out1(train_y),
                                 batch_size=batch_size, epochs=20,
                                 validation_data=([val_x,val_mask.reshape(-1,1000,1)], getTrue4out1(val_y)),
                                 callbacks=[checkpoint_acc],verbose=1)

In [6]:
 '''我们常常可以把argparse的使用简化成下面四个步骤
       1：import argparse
       2：parser = argparse.ArgumentParser()
       3：parser.add_argument()
       4：parser.parse_args()
       上面四个步骤解释如下：首先导入该模块；然后创建一个解析对象；然后向该对象中添加你要关注的命令行参数和选项，
       每一个add_argument方法对应一个你要关注的参数或选项；最后调用parse_args()方法进行解析；解析成功之后即可使用'''
    
def main():
    #加default
    # description= 这个参数简要描述这个程度做什么以及怎么做
    parser=argparse.ArgumentParser(
        description='MULocDeep: interpretable protein localization classifier at sub-cellular and sub-organellar levels')
    #MULocDeep_model  UniLoc-train-20nr
    #--lv1_input_dir/--lv2_input_dir 亚细胞训练数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--lv1_input_dir', dest='lv1_dir', type=str, 
                        default="D:/mulocdeep/mul_data",
                        help='sub-cellular training data, contains 8 folds protein sequences and labels', required=False)
    parser.add_argument('--lv2_input_dir', dest='lv2_dir', type=str,
                       default="D:/mulocdeep/mul_data",
                       help='sub-cellular training data, contains 8 folds protein sequences and labels', required=False)
    #--MULocDeep_model 添加它来训练MULocDeep模型，否则训练一个var模型
    parser.add_argument('--MULocDeep_model', dest='modeltype', action='store_false',  #触发，store_true会触发DeepLoc
                        #如果是store_false,则默认值是True，如果是store_true,则默认值是False  
                        help='Add this to train the MULocDeep model, otherwise train a variant model', required=False)
    #--model_output 受过训练的模型存储的目录的名称  需自己添加
    parser.add_argument('--model_output', dest='outputdir', type=str, 
                       default="D:/mulocdeep/lv1_result3",
                       help='the name of the directory where the trained model stores', required=False)  #由True改成False
    
    parser.add_argument('-existPSSM', dest='existPSSM', type=str,
                        default="D:/mulocdeep/mulocdeep_pssm",
                        help='the name of the existing PSSM directory if there is one.', required=False)
    
    #var_model  deeploc_40nr_8folds
    #--input_dir 训练var模型的数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--input_var', dest='var_dir', type=str,
                        default="D:/deeploc/deeploc_40nr_8folds",
                        help='data for traing the variant model, contains 8 folds protein sequences and labels', required=False)
    #改true  并且还需要加一个model_ouput  一个是deeploc  一个是MULocDeep
    parser.add_argument('--var_model_output', dest='var_outputdir', type=str, help='the name of the directory where the trained model stores', 
                        default="D:/deeploc/var_model_result1",
                        required=False)  #由True改成False
    parser.add_argument('-var_existPSSM', dest='var_existPSSM', type=str,
                        default="D:/deeploc/deeploc_pssm",
                        help='the name of the existing PSSM directory if there is one.', required=False)
    parser.set_defaults(feature=True)
    #args = parser.parse_args()   #改
    args = parser.parse_known_args()[0]   #jupyter下运行解析需要此代码
    model_type=args.modeltype
    input_lv1=args.lv1_dir
    input_lv2 = args.lv2_dir
    outputdir=args.outputdir
    existPSSM = args.existPSSM
    input_var=args.var_dir
    var_outputdir=args.var_outputdir
    var_existPSSM = args.var_existPSSM

    if model_type==True:
        if not input_lv1[len(input_lv1) - 1] == "/":
            input_lv1 = input_lv1 + "/"
        if not input_lv2[len(input_lv2) - 1] == "/":
            input_lv2 = input_lv2 + "/"
        if not outputdir[len(outputdir) - 1] == "/":
            outputdir = outputdir + "/"
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)
        if existPSSM != "":
            if not existPSSM[len(existPSSM) - 1] == "/":
                existPSSM = existPSSM + "/"
        if ((existPSSM == "") or (not os.path.exists(existPSSM))):
            ts = calendar.timegm(time.gmtime())
            pssmdir = outputdir + str(ts) + "_pssm/"
            if not os.path.exists(pssmdir):
                os.makedirs(pssmdir)
            process_input_train(input_lv1 + "lv1_train.txt", pssmdir)
            process_input_train(input_lv2 + "lv2_train.txt", pssmdir)
            for foldnum in range(8):
                train_MULocDeep(input_lv1, input_lv2, pssmdir, outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_MULocDeep(input_lv1, input_lv2, existPSSM, outputdir, foldnum)
    elif model_type==False:
        if not input_var[len(input_var) - 1] == "/":
            input_var = input_var + "/"
        if not var_outputdir[len(var_outputdir) - 1] == "/":
            var_outputdir = var_outputdir + "/"
        if not os.path.exists(var_outputdir):
            os.mkdir(var_outputdir)
        if existPSSM != "":
            if not var_existPSSM[len(var_existPSSM) - 1] == "/":
                var_existPSSM = var_existPSSM + "/"
        if ((var_existPSSM == "") or (not os.path.exists(var_existPSSM))):
            ts = calendar.timegm(time.gmtime())
            pssmdir = var_outputdir + str(ts) + "_pssm/"
            if not os.path.exists(pssmdir):
                os.makedirs(pssmdir)
            process_input_train(input_var + "processed_deeploc_train_S_seq", pssmdir)
            for foldnum in range(8):
                train_var(input_var, pssmdir, var_outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_var(input_var, var_existPSSM, var_outputdir, foldnum)

In [7]:
if __name__ == "__main__":
    main()

doing 0th fold
Instructions for updating:
Colocations handled automatically by placer.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 1000, 25)     0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1000, 25)     0           Input[0][0]                      
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1000, 1)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1000, 25)     0           dropout_1[0][0]                  
     

Instructions for updating:
Use tf.cast instead.
Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 141s 8ms/step - loss: 0.7218 - accuracy: 0.7187 - val_loss: 0.6630 - val_accuracy: 0.8265

Epoch 00001: val_accuracy improved from -inf to 0.82650, saving model to D:/mulocdeep/lv1_result3/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.66300, saving model to D:/mulocdeep/lv1_result3/fold0_big_lv1_loss-weights.hdf5
epoch 1

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 112s 7ms/step - loss: 0.6338 - accuracy: 0.8203 - val_loss: 0.6117 - val_accuracy: 0.8236

Epoch 00001: val_accuracy did not improve from 0.82650

Epoch 00001: val_loss improved from 0.66300 to 0.61173, saving model to D:/mulocdeep/lv1_result3/fold0_big_lv1_loss-weights.hdf5
epoch 2

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [============================


Epoch 00001: val_accuracy did not improve from 0.93010

Epoch 00001: val_loss improved from 0.32933 to 0.32101, saving model to D:/mulocdeep/lv1_result3/fold0_big_lv1_loss-weights.hdf5
epoch 18

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 109s 6ms/step - loss: 0.2785 - accuracy: 0.9469 - val_loss: 0.3326 - val_accuracy: 0.9235

Epoch 00001: val_accuracy did not improve from 0.93010

Epoch 00001: val_loss did not improve from 0.32101
epoch 19

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 116s 7ms/step - loss: 0.2682 - accuracy: 0.9484 - val_loss: 0.3042 - val_accuracy: 0.9297

Epoch 00001: val_accuracy did not improve from 0.93010

Epoch 00001: val_loss improved from 0.32101 to 0.30424, saving model to D:/mulocdeep/lv1_result3/fold0_big_lv1_loss-weights.hdf5
doing 1th fold
Model: "model_4"
____________________________________________________________________________

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 127s 7ms/step - loss: 0.7293 - accuracy: 0.7084 - val_loss: 0.6588 - val_accuracy: 0.8146

Epoch 00001: val_accuracy improved from -inf to 0.81460, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.65881, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_loss-weights.hdf5
epoch 1

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 120s 7ms/step - loss: 0.6335 - accuracy: 0.8233 - val_loss: 0.6251 - val_accuracy: 0.8430

Epoch 00001: val_accuracy improved from 0.81460 to 0.84303, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.65881 to 0.62512, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_loss-weights.hdf5
epoch 2

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 


Epoch 00001: val_accuracy did not improve from 0.92172

Epoch 00001: val_loss improved from 0.33395 to 0.33307, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_loss-weights.hdf5
epoch 19

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 118s 7ms/step - loss: 0.2645 - accuracy: 0.9496 - val_loss: 0.3154 - val_accuracy: 0.9247

Epoch 00001: val_accuracy improved from 0.92172 to 0.92466, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.33307 to 0.31545, saving model to D:/mulocdeep/lv1_result3/fold1_big_lv1_loss-weights.hdf5
doing 2th fold
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 1000, 25)     0                                            
________________

...

Train on 17241 samples, validate on 2342 samples
Epoch 1/1
17241/17241 [==============================] - 123s 7ms/step - loss: 0.7214 - accuracy: 0.7314 - val_loss: 0.6738 - val_accuracy: 0.8172

Epoch 00001: val_accuracy improved from -inf to 0.81721, saving model to D:/mulocdeep/lv1_result3/fold5_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.67378, saving model to D:/mulocdeep/lv1_result3/fold5_big_lv1_loss-weights.hdf5
epoch 1

Train on 17241 samples, validate on 2342 samples
Epoch 1/1
17241/17241 [==============================] - 130s 8ms/step - loss: 0.6291 - accuracy: 0.8320 - val_loss: 0.6280 - val_accuracy: 0.8254

Epoch 00001: val_accuracy improved from 0.81721 to 0.82541, saving model to D:/mulocdeep/lv1_result3/fold5_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.67378 to 0.62796, saving model to D:/mulocdeep/lv1_result3/fold5_big_lv1_loss-weights.hdf5
epoch 2

Train on 17241 samples, validate on 2342 samples
Epoch 1/1
17241/17241 

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 122s 7ms/step - loss: 0.7293 - accuracy: 0.7110 - val_loss: 0.6728 - val_accuracy: 0.8202

Epoch 00001: val_accuracy improved from -inf to 0.82015, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.67283, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_loss-weights.hdf5
epoch 1

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 113s 7ms/step - loss: 0.6346 - accuracy: 0.8294 - val_loss: 0.6040 - val_accuracy: 0.8551

Epoch 00001: val_accuracy improved from 0.82015 to 0.85515, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.67283 to 0.60398, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_loss-weights.hdf5
epoch 2

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 


Epoch 00001: val_loss improved from 0.33140 to 0.31310, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_loss-weights.hdf5
epoch 19

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 101s 6ms/step - loss: 0.2683 - accuracy: 0.9490 - val_loss: 0.3046 - val_accuracy: 0.9291

Epoch 00001: val_accuracy did not improve from 0.92925

Epoch 00001: val_loss improved from 0.31310 to 0.30460, saving model to D:/mulocdeep/lv1_result3/fold6_big_lv1_loss-weights.hdf5
doing 7th fold
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 1000, 25)     0                                            
__________________________________________________________________________________________________
dropout_36 (Dropout)            (None, 1000, 25) 

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 115s 7ms/step - loss: 0.7340 - accuracy: 0.7012 - val_loss: 0.6611 - val_accuracy: 0.8203

Epoch 00001: val_accuracy improved from -inf to 0.82028, saving model to D:/mulocdeep/lv1_result3/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 0.66111, saving model to D:/mulocdeep/lv1_result3/fold7_big_lv1_loss-weights.hdf5
epoch 1

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 106s 6ms/step - loss: 0.6346 - accuracy: 0.8309 - val_loss: 0.6189 - val_accuracy: 0.8411

Epoch 00001: val_accuracy improved from 0.82028 to 0.84111, saving model to D:/mulocdeep/lv1_result3/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 0.66111 to 0.61894, saving model to D:/mulocdeep/lv1_result3/fold7_big_lv1_loss-weights.hdf5
epoch 2

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 


Epoch 00001: val_accuracy did not improve from 0.92929

Epoch 00001: val_loss improved from 0.31796 to 0.31473, saving model to D:/mulocdeep/lv1_result3/fold7_big_lv1_loss-weights.hdf5
epoch 19

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 102s 6ms/step - loss: 0.2664 - accuracy: 0.9488 - val_loss: 0.3184 - val_accuracy: 0.9249

Epoch 00001: val_accuracy did not improve from 0.92929

Epoch 00001: val_loss did not improve from 0.31473
